In [ ]:
# Import the drive module from google.colab
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')



```
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

```



In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, SimpleRNN, LSTM, TimeDistributed
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import random

# Define the image size
IMG_SIZE = 224

# Define directories
base_dir = '/content/drive/MyDrive/Food Detection.v1-food-detection.retinanet (1)/'
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# Load annotations
def load_annotations(directory):
    annotations_path = os.path.join(directory, '_annotations.csv')
    if os.path.exists(annotations_path):
        annotations = pd.read_csv(annotations_path, header=None, sep=",")
        print(f"First few rows of annotations:\n{annotations.head()}")
        annotations.columns = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class']
        return annotations
    else:
        raise FileNotFoundError(f"No annotation file found in directory {directory}")

train_annotations = load_annotations(train_dir)
valid_annotations = load_annotations(valid_dir)
test_annotations = load_annotations(test_dir)

# Ensure labels are treated as strings
train_annotations['class'] = train_annotations['class'].astype(str)
valid_annotations['class'] = valid_annotations['class'].astype(str)
test_annotations['class'] = test_annotations['class'].astype(str)

# Create a combined set of labels from all datasets
all_labels = set(train_annotations['class']).union(set(valid_annotations['class'])).union(set(test_annotations['class']))

# Create a label map based on all unique labels
label_names = sorted(all_labels)
label_map = {name: index for index, name in enumerate(label_names)}



```
First few rows of annotations:
                                                   0    1    2    3    4  \
0   6219_jpg.rf.f2e1f1588186c338271cde6bb6d39591.jpg  169    0  415  339   
1  36042_jpg.rf.f01a1e0ef9bb3a183f82590c08d61cdf.jpg  202    1  387  191   
2  307941_jpg.rf.f0aa456a97c4f080d6fa78194085e411...    0   96  151  244   
3  307941_jpg.rf.f0aa456a97c4f080d6fa78194085e411...   87  130  283  344   
4  307941_jpg.rf.f0aa456a97c4f080d6fa78194085e411...  285  106  415  387   

            5  
0  Cheesecake  
1       Steak  
2    Macarons  
3    Macarons  
4    Macarons  
First few rows of annotations:
                                                   0    1    2    3    4  \
0  232335_jpg.rf.00120909e356a1850422e77551141428...   21  101  326  378   
1  176508_jpg.rf.189bc4a850a5e4c8470731b57c66a240...   95  198  179  351   
2  176508_jpg.rf.189bc4a850a5e4c8470731b57c66a240...  159  181  362  390   
3  148361_jpg.rf.0e115e9a46a2761676e2d2a05ae4a61a...  297   27  374  200   
4  148361_jpg.rf.0e115e9a46a2761676e2d2a05ae4a61a...    8   21   87  224   

          5  
0  Tiramisu  
1     Steak  
2     Steak  
3  Macarons  
4  Macarons  
First few rows of annotations:
                                                   0    1    2    3    4  \
0  248448_jpg.rf.55349c6550edbabfcfff10a8bcc49b5a...   56   69  331  348   
1  129895_jpg.rf.75dd03e871714ec9250fed6a567f1c31...  310   79  376  180   
2  129895_jpg.rf.75dd03e871714ec9250fed6a567f1c31...  261   86  328  203   
3  129895_jpg.rf.75dd03e871714ec9250fed6a567f1c31...  215  104  281  207   
4  129895_jpg.rf.75dd03e871714ec9250fed6a567f1c31...  166  120  225  235   

               5  
0     Cheesecake  
1       Macarons  
2  Macarons-Pink  
3       Macarons  
4   Macarons-Red  
```

